
# 🧠 String Interview Playbook — do básico ao avançado

Este notebook reúne **problemas clássicos e alguns avançados de strings**, cada um com **3 versões**:

- **Newbie**: direta, didática — às vezes menos eficiente.
- **Fodona**: solução ótima/algorítmica (janela deslizante, DP, KMP, Manacher, etc.).
- **Pythônica apelona**: usando o poder da linguagem (ex.: `Counter`, `sorted`, `re`) — *quando a entrevista permitir*.

Para cada problema há **explicação**, **complexidade** e **perguntas típicas** de entrevista.


In [62]:

def run_tests(func, cases):
    for i, (args, expected) in enumerate(cases, 1):
        if not isinstance(args, tuple):
            args = (args,)
        got = func(*args)
        assert got == expected, f"case {i} failed: got={got}, expected={expected}, args={args}"
    "✓ all tests passed"



## 1) Reverter string

**Tarefa**: dado `s`, retorne `s` invertida.

**Discussão**  
- Strings são imutáveis → converta para lista e troque extremidades.
- Entrevistador pode pedir in-place: em `str` não dá; em `list` dá.

**Complexidade**: `O(n)` tempo.


### Newbie

- **O que é:** implementação didática para mostrar como inverter caractere a caractere.
- **Ideia:** copiar a string para uma lista mutável e trocar os extremos com dois ponteiros até eles se cruzarem.
- **Complexidade:** tempo `O(n)`; espaço `O(n)` pela cópia da lista antes de juntar de volta.


In [35]:

def reverse_string_newbie(s: str) -> str:
    a = list(s)
    i, j = 0, len(a) - 1
    while i < j:
        a[i], a[j] = a[j], a[i]
        i += 1; j -= 1
    return "".join(a)

# quick tests
assert reverse_string_newbie("abc") == "cba"


### Fodona

- **O que é:** variação em estilo de entrevista usando os mesmos dois ponteiros com nomenclatura mais padrão.
- **Ideia:** mover `l` e `r` em direção ao centro trocando posições simetricamente, sem criar estruturas extras além da lista temporária.
- **Complexidade:** tempo `O(n)`; espaço `O(n)` por causa da lista criada a partir da string imutável.


In [36]:

def reverse_string_fodona(s: str) -> str:
    a = list(s)
    l, r = 0, len(a) - 1
    while l < r:
        a[l], a[r] = a[r], a[l]
        l += 1; r -= 1
    return "".join(a)


### Pythônica apelona

- **O que é:** usar slicing com passo negativo para delegar o trabalho ao runtime.
- **Ideia:** `s[::-1]` percorre a string uma vez em ordem reversa e constrói a cópia diretamente.
- **Complexidade:** tempo `O(n)`; espaço `O(n)` porque uma nova string é criada.


In [37]:

def reverse_string_pythonica(s: str) -> str:
    return s[::-1]



## 2) Verificar anagramas sem `sorted`


### Newbie

- **O que é:** comparação de anagramas contando frequências com um `dict`.
- **Ideia:** incrementar a contagem de cada caractere de `a` e decrementar com `b`, falhando se alguma contagem fica negativa.
- **Complexidade:** tempo `O(n)`; espaço `O(k)` onde `k` é o número de caracteres distintos na string.


In [38]:

def is_anagram_newbie(a: str, b: str) -> bool:
    if len(a) != len(b): return False
    cnt = {}
    for ch in a:
        cnt[ch] = cnt.get(ch, 0) + 1
    for ch in b:
        if cnt.get(ch, 0) == 0:
            return False
        cnt[ch] -= 1
    return True


### Fodona (array 26)

- **O que é:** otimização para letras minúsculas usando um vetor fixo de 26 posições.
- **Ideia:** mapear `a..z` para índices do array, atualizando contagens e abortando se aparecer caractere fora desse domínio.
- **Complexidade:** tempo `O(n)`; espaço `O(1)` porque o array tem tamanho fixo.


In [39]:

def is_anagram_fodona(a: str, b: str) -> bool:
    if len(a) != len(b): return False
    base = ord('a')
    freq = [0]*26
    for ch in a:
        idx = ord(ch)-base
        if 0 <= idx < 26:
            freq[idx]+=1
        else:
            return is_anagram_newbie(a,b)
    for ch in b:
        idx = ord(ch)-base
        if 0 <= idx < 26:
            freq[idx]-=1
            if freq[idx]<0: return False
        else:
            return is_anagram_newbie(a,b)
    return all(v==0 for v in freq)


### Pythônica apelona

- **O que é:** solução de alto nível com `collections.Counter`.
- **Ideia:** construir dois contadores e comparar objetos, deixando o C otimizar o loop interno.
- **Complexidade:** tempo `O(n)`; espaço `O(k)` para armazenar as contagens de cada caractere.


In [40]:

from collections import Counter
def is_anagram_pythonica(a: str, b: str) -> bool:
    return Counter(a) == Counter(b)



## 3) Primeiro caractere não repetido


### Newbie

- **O que é:** localizar o primeiro caractere único com duas passadas e um `dict`.
- **Ideia:** primeira passada soma frequências; segunda encontra o primeiro índice com contagem igual a 1.
- **Complexidade:** tempo `O(n)`; espaço `O(k)` para guardar as contagens.


In [41]:

def first_unique_index_newbie(s: str) -> int:
    cnt = {}
    for ch in s:
        cnt[ch] = cnt.get(ch, 0) + 1
    for i, ch in enumerate(s):
        if cnt[ch] == 1:
            return i
    return -1


### Fodona (26)

- **O que é:** mesma ideia restrita a letras minúsculas, evitando dicionários dinâmicos.
- **Ideia:** registrar posição e frequência em dois arrays de 26 elementos e no fim buscar o menor índice com contagem 1.
- **Complexidade:** tempo `O(n)`; espaço `O(1)` ao manter arrays de tamanho fixo.


In [42]:

def first_unique_index_fodona(s: str) -> int:
    first = [-1]*26
    count = [0]*26
    base = ord('a')
    for i, ch in enumerate(s):
        idx = ord(ch)-base
        if 0 <= idx < 26:
            if first[idx] == -1: first[idx] = i
            count[idx] += 1
        else:
            return first_unique_index_newbie(s)
    ans = 10**9
    for k in range(26):
        if count[k]==1: ans = min(ans, first[k])
    return -1 if ans==10**9 else ans


### Pythônica apelona

- **O que é:** abordagem expressiva usando `Counter`.
- **Ideia:** contar tudo de uma vez e fazer o segundo laço lendo do contador pré-computado.
- **Complexidade:** tempo `O(n)`; espaço `O(k)` para o dicionário de frequências.


In [43]:

from collections import Counter
def first_unique_index_pythonica(s: str) -> int:
    cnt = Counter(s)
    for i,ch in enumerate(s):
        if cnt[ch]==1: return i
    return -1



## 4) Palíndromo válido (ignora não-alfaNum; case-insensitive)


### Duas pontas

- **O que é:** checagem de palíndromo realista ignorando símbolos e capitalização.
- **Ideia:** ponteiros `i` e `j` avançam enquanto pulam caracteres não alfanuméricos; quando ambos param, comparam versões minúsculas.
- **Complexidade:** tempo `O(n)`; espaço `O(1)` porque só usamos índices.


In [44]:

def is_palindrome_valid(s: str) -> bool:
    i, j = 0, len(s)-1
    while i < j:
        while i < j and not s[i].isalnum(): i+=1
        while i < j and not s[j].isalnum(): j-=1
        if s[i].lower() != s[j].lower(): return False
        i+=1; j-=1
    return True


### Pythônica apelona

- **O que é:** solução declarativa baseada em compreensões.
- **Ideia:** filtrar apenas caracteres alfanuméricos minúsculos, gerar uma lista e compará-la com sua reversa.
- **Complexidade:** tempo `O(n)`; espaço `O(n)` para armazenar a lista filtrada.


In [45]:

def is_palindrome_pythonica(s: str) -> bool:
    t = [c.lower() for c in s if c.isalnum()]
    return t == t[::-1]



## 5) Maior substring sem repetir caracteres


### Newbie (set + move)

- **O que é:** implementação direta da janela deslizante com um `set`.
- **Ideia:** expandir `j` enquanto os caracteres são únicos; quando repete, remover `s[i]` e avançar `i`.
- **Complexidade:** tempo `O(n)`; espaço `O(k)` onde `k` é o tamanho da janela sem repetição.


In [46]:

def length_longest_substring_newbie(s: str) -> int:
    seen = set(); i=j=best=0
    while i<len(s) and j<len(s):
        if s[j] not in seen:
            seen.add(s[j]); j+=1; best=max(best, j-i)
        else:
            seen.remove(s[i]); i+=1
    return best


### Fodona (último índice)

- **O que é:** janela deslizante mais eficiente rastreando a última posição de cada caractere.
- **Ideia:** se `ch` reaparece dentro da janela atual, movemos o início para `last[ch] + 1` e atualizamos o melhor comprimento.
- **Complexidade:** tempo `O(n)`; espaço `O(k)` para o mapa de último índice.


In [47]:

def length_longest_substring_fodona(s: str) -> int:
    last = {}; start = best = 0
    for i,ch in enumerate(s):
        if ch in last and last[ch] >= start:
            start = last[ch]+1
        last[ch] = i
        if i-start+1 > best: best = i-start+1
    return best


### Pythônica apelona (compacta)

- **O que é:** versão pythonizada da estratégia de último índice.
- **Ideia:** usar `dict.get` com sentinela para condensar o update do início em uma única linha.
- **Complexidade:** tempo `O(n)`; espaço `O(k)` para o dicionário de posições.


In [48]:

def length_longest_substring_pythonica(s: str) -> int:
    last = {}; start = best = 0
    for i,ch in enumerate(s):
        start = max(start, last.get(ch,-1)+1)
        last[ch] = i
        best = max(best, i-start+1)
    return best



## 6) Substring search (KMP) — sem `find`


### Fodona (KMP)

- **O que é:** Knuth–Morris–Pratt, o algoritmo linear clássico para busca de padrões.
- **Ideia:** pré-calcular o array `lps` com o maior prefixo também sufixo e usar esse conhecimento para não voltar `i` ao tratar mismatches.
- **Complexidade:** tempo `O(n + m)`; espaço `O(m)` para armazenar o vetor `lps`.


In [49]:

def kmp_search(text: str, pat: str) -> int:
    if pat=="": return 0
    lps = [0]*len(pat)
    i=j=1; j=0
    while i < len(pat):
        if pat[i]==pat[j]:
            j+=1; lps[i]=j; i+=1
        elif j:
            j=lps[j-1]
        else:
            lps[i]=0; i+=1
    i=j=0
    while i < len(text):
        if text[i]==pat[j]:
            i+=1; j+=1
            if j==len(pat): return i-j
        elif j:
            j=lps[j-1]
        else:
            i+=1
    return -1


### Newbie (força bruta)

- **O que é:** comparação direta de cada janela possível no texto.
- **Ideia:** alinhar o padrão em todas as posições e avançar caracter por caracter até falhar ou encontrar o match completo.
- **Complexidade:** tempo `O(n·m)`; espaço `O(1)` sem estruturas auxiliares.


In [50]:

def find_bruteforce(text: str, pat: str) -> int:
    if pat=="": return 0
    n,m=len(text),len(pat)
    for i in range(n-m+1):
        k=0
        while k<m and text[i+k]==pat[k]:
            k+=1
        if k==m: return i
    return -1


### Pythônica apelona

- **O que é:** delegar a busca ao método `str.find`.
- **Ideia:** confiar na implementação otimizada em C do Python para realizar a busca.
- **Complexidade:** tempo `O(n·m)` no pior caso; espaço `O(1)` além da string original.


In [51]:

def find_pythonica(text: str, pat: str) -> int:
    return text.find(pat)



## 7) Maior substring palindrômica


### Newbie (expandir centros)

- **O que é:** estratégia clássica de expandir palíndromos a partir de cada posição.
- **Ideia:** tentar centros ímpares e pares via função `expand`, guardando a melhor faixa encontrada.
- **Complexidade:** tempo `O(n^2)`; espaço `O(1)` porque usamos apenas índices.


In [52]:

def longest_pal_substring_newbie(s: str) -> str:
    if not s: return ""
    def expand(l,r):
        while l>=0 and r<len(s) and s[l]==s[r]:
            l-=1; r+=1
        return l+1, r
    best=(0,1)
    for i in range(len(s)):
        l1,r1=expand(i,i)
        l2,r2=expand(i,i+1)
        if r1-l1>r2-l2 and r1-l1>best[1]-best[0]: best=(l1,r1)
        elif r2-l2>best[1]-best[0]: best=(l2,r2)
    return s[best[0]:best[1]]


### Fodona (Manacher)

- **O que é:** algoritmo Manacher para encontrar o maior palíndromo em tempo linear.
- **Ideia:** inserir separadores, espelhar raios já conhecidos e expandir quando possível para cada posição.
- **Complexidade:** tempo `O(n)`; espaço `O(n)` para o array auxiliar de raios.


In [53]:

def longest_pal_substring_manacher(s: str) -> str:
    if not s: return ""
    t=['^']
    for ch in s: t+=['#',ch]
    t+=['#','$']
    n=len(t); p=[0]*n; c=r=0
    for i in range(1,n-1):
        mir=2*c-i
        if i<r: p[i]=min(r-i,p[mir])
        while t[i+1+p[i]]==t[i-1-p[i]]: p[i]+=1
        if i+p[i]>r: c,r=i,i+p[i]
    max_len=center=max((p[i],i) for i in range(1,n-1))
    start=(center-max_len)//2
    return s[start:start+max_len]


### Pythônica apelona (compacta de centros)

- **O que é:** versão enxuta da expansão de centros, privilegiando legibilidade.
- **Ideia:** iterar sobre centros possíveis com dupla atribuição `(i,i)` e `(i,i+1)` e atualizar a melhor substring em linha.
- **Complexidade:** tempo `O(n^2)`; espaço `O(1)` além das variáveis de apoio.


In [54]:

def longest_pal_substring_pythonica(s: str) -> str:
    best=""
    for i in range(len(s)):
        for a,b in ((i,i),(i,i+1)):
            l,r=a,b
            while l>=0 and r<len(s) and s[l]==s[r]:
                if r-l+1>len(best): best=s[l:r+1]
                l-=1; r+=1
    return best



## 8) Menor janela contendo `t` em `s`


### Fodona (janela)

- **O que é:** solução ótima para a menor janela que cobre `t`.
- **Ideia:** manter contagens em `need`, expandir `j` até cobrir tudo e depois contrair `i` enquanto possível para minimizar.
- **Complexidade:** tempo `O(n)`; espaço `O(k)` para o mapa de caracteres necessários.


In [55]:

def min_window(s: str, t: str) -> str:
    if not s or not t: return ""
    need={}
    for ch in t: need[ch]=need.get(ch,0)+1
    missing=len(t); i=start=end=0
    for j,ch in enumerate(s,1):
        if need.get(ch,0)>0: missing-=1
        need[ch]=need.get(ch,0)-1
        while missing==0:
            if end==0 or j-i<end-start: start,end=i,j
            need[s[i]]=need.get(s[i],0)+1
            if need[s[i]]>0: missing+=1
            i+=1
    return s[start:end]


### Newbie (brute force educativo)

- **O que é:** abordagem exaustiva útil para entender o problema antes da otimização.
- **Ideia:** testar todas as substrings de `s` e verificar com um contador auxiliar se cobrem `t`.
- **Complexidade:** tempo `O(n^2·k)`; espaço `O(k)` para o mapa temporário usado em cada checagem.


In [56]:

def min_window_newbie(s: str, t: str) -> str:
    if not s or not t: return ""
    def covers(a: str, t: str) -> bool:
        need={}
        for ch in t: need[ch]=need.get(ch,0)+1
        for ch in a:
            if need.get(ch,0)>0: need[ch]-=1
        return all(v==0 for v in need.values())
    best=""
    for i in range(len(s)):
        for j in range(i+1,len(s)+1):
            sub=s[i:j]
            if (not best or len(sub)<len(best)) and covers(sub,t):
                best=sub
    return best


### Pythônica apelona (Counter)

- **O que é:** variante pythonizada da janela deslizante usando `Counter`.
- **Ideia:** `Counter(t)` inicializa as necessidades; o loop principal decrementa/ incrementa contagens enquanto move a janela.
- **Complexidade:** tempo `O(n)`; espaço `O(k)` para o contador.


In [57]:

from collections import Counter
def min_window_pythonica(s: str, t: str) -> str:
    need=Counter(t); missing=len(t)
    i=start=end=0
    for j,ch in enumerate(s,1):
        if need[ch]>0: missing-=1
        need[ch]-=1
        while missing==0:
            if end==0 or j-i<end-start: start,end=i,j
            need[s[i]]+=1
            if need[s[i]]>0: missing+=1
            i+=1
    return s[start:end]



## 9) Decodificar `"3[a2[c]]"`


### Fodona (pilhas)

- **O que é:** decodificador iterativo para strings com padrões `k[substr]`.
- **Ideia:** empilhar contagens e prefixos toda vez que encontramos um `[` e, ao fechar `]`, repetir o trecho atual `k` vezes e concatenar com o prefixo anterior.
- **Complexidade:** tempo `O(n)`; espaço `O(n)` no pior caso para as pilhas quando há muitos blocos aninhados.


In [58]:

def decode_string(s: str) -> str:
    num_st, str_st = [], []
    cur=[]; k=0
    for ch in s:
        if ch.isdigit():
            k=k*10+(ord(ch)-48)
        elif ch=='[':
            num_st.append(k); str_st.append(cur); k=0; cur=[]
        elif ch==']':
            rep=num_st.pop(); prev=str_st.pop()
            cur = prev + cur*rep
        else:
            cur.append(ch)
    return "".join(cur)



## 10) Distância de edição (Levenshtein)


### Newbie (tabela completa)

- **O que é:** implementação clássica da distância de edição usando matriz cheia.
- **Ideia:** preencher tabela `dp` de `(n+1)×(m+1)` aplicando inserção, remoção e substituição com recursão dinâmica.
- **Complexidade:** tempo `O(n·m)`; espaço `O(n·m)` para a matriz completa.


In [59]:

def edit_distance_newbie(a: str, b: str) -> int:
    n,m=len(a),len(b)
    dp=[[0]*(m+1) for _ in range(n+1)]
    for i in range(n+1): dp[i][0]=i
    for j in range(m+1): dp[0][j]=j
    for i in range(1,n+1):
        for j in range(1,m+1):
            cost=0 if a[i-1]==b[j-1] else 1
            dp[i][j]=min(dp[i-1][j]+1, dp[i][j-1]+1, dp[i-1][j-1]+cost)
    return dp[n][m]


### Fodona (duas linhas)

- **O que é:** otimização de espaço da DP mantendo apenas a linha anterior.
- **Ideia:** iterar sobre `a` e `b`, atualizando um vetor corrente e reaproveitando o vetor anterior para calcular o custo mínimo.
- **Complexidade:** tempo `O(n·m)`; espaço `O(min(n, m))` graças às duas linhas.


In [60]:

def edit_distance_fodona(a: str, b: str) -> int:
    if len(a) < len(b): a,b=b,a
    prev=list(range(len(b)+1))
    for i,ca in enumerate(a,1):
        cur=[i]
        for j,cb in enumerate(b,1):
            cost=0 if ca==cb else 1
            cur.append(min(prev[j]+1, cur[j-1]+1, prev[j-1]+cost))
        prev=cur
    return prev[-1]


### Pythônica apelona

- **O que é:** lembrete de que não existe atalho builtin para Levenshtein em Python puro.
- **Ideia:** reutilizar a versão `fodona` ou bibliotecas externas; o truque aqui é explicar ao entrevistador a limitação.
- **Complexidade:** tempo `O(n·m)`; espaço `O(min(n, m))` se reaproveitarmos a DP otimizada.


In [61]:

# Não há builtin — a versão 'fodona' já é ideal em Python puro.



---
## Perguntas típicas e como responder

- **Por que sua solução é `O(n)`?**  
  Explico que percorro cada caractere no máximo `k` vezes e mantenho mapas de tamanho limitado.

- **Pode otimizar espaço?**  
  Sim — duas pontas (O(1)) ou DP em duas linhas para Levenshtein.

- **E Unicode?**  
  Trocar array fixo por `dict` e considerar `casefold()`. Para remoção de acentos, normalizar com `unicodedata` (se permitido).

- **Edge cases?**  
  Strings vazias, um caractere, todos iguais, padrão no início/fim, entradas grandes.
